In [1]:
import numpy as np
import cv2
from itertools import cycle
import pickle
import pathlib
import math
import tqdm
import scipy.io
from matplotlib import pyplot as plt
import scipy.io
import h5py
import re
from lxml import etree as ET
import scipy.signal as sig
import pandas as pd
from scipy.stats import kde
from BlockSync_current import BlockSync
import UtilityFunctions_newOE as uf
from scipy import signal
import bokeh
import seaborn as sns

In [2]:
def create_distance_plot(distances, top_dist_to_show=500):
    # Create cumulative distribution plot
    sns.set(style="whitegrid")
    fig, axs = plt.subplots(2, figsize=(6, 6), dpi=150)
    
    axs[0].set_title('Cumulative Euclidean Distances for Camera Jitter', fontsize=15)
    axs[0].set_ylabel('Cumulative \n % of Frames')
    axs[0].set_xlim(0, top_dist_to_show)
    axs[0].grid(False)
    
    # Create histogram and cumulative distribution
    sns.kdeplot(distances, cumulative=True, label='Left Eye', ax=axs[0], linewidth=4, c='black')
    
    axs[1].hist(distances, bins=np.linspace(0, top_dist_to_show, 20), log=False, color='black')

    # Set title and labels
    title = 'Image displacement histogram'
    axs[1].set_title(title, fontsize=15)
    axs[1].set_xlabel('Euclidean Displacement [$\mu$m]', fontsize=15)
    axs[1].set_xscale('linear')
    axs[1].set_yscale('linear')
    axs[1].set_ylabel('Frame count', fontsize=15)

    # Adjust tick label sizes
    axs[1].tick_params(axis='x', which='major', labelsize=15)

    # Set white background and black text
    axs[1].set_facecolor('white')
    axs[1].title.set_color('black')
    axs[1].xaxis.label.set_color('black')
    axs[1].yaxis.label.set_color('black')
    axs[1].tick_params(colors='black')
    axs[1].grid(False)

    plt.tight_layout()

    return fig, axs
def add_intermediate_elements(input_vector, gap_to_bridge):
    # Step 1: Calculate differences between each element
    differences = np.diff(input_vector)

    # Step 2: Add intervening elements based on the diff_threshold
    output_vector = [input_vector[0]]
    for i, diff in enumerate(differences):
        if diff < gap_to_bridge:
            # Add intervening elements
            output_vector.extend(range(input_vector[i] + 1, input_vector[i + 1]))

        # Add the next element from the original vector
        output_vector.append(input_vector[i + 1])

    return np.sort(np.unique(output_vector))

def find_jittery_frames(block, eye, max_distance, diff_threshold, gap_to_bridge=6):
    
    #input checks
    if eye not in ['left', 'right']:
        print(f'eye can only be left/right, your input: {eye}')
        return None
    # eye setup
    if eye == 'left':
        jitter_dict = block.le_jitter_dict
        eye_frame_col = 'L_eye_frame'
    elif eye == 'right':
        jitter_dict = block.re_jitter_dict
        eye_frame_col = 'R_eye_frame'
    
    df_dict = {'left':block.le_df,
               'right':block.re_df}
    
    df = pd.DataFrame.from_dict(jitter_dict)
    indices_of_highest_drift = df.query("top_correlation_dist > @max_distance").index.values
    diff_vec = np.diff(df['top_correlation_dist'].values)
    diff_peaks_indices = np.where(diff_vec > diff_threshold)[0]
    video_indices = np.concatenate((diff_peaks_indices, indices_of_highest_drift))
    print(f'the diff based jitter frame exclusion gives: {np.shape(diff_peaks_indices)}')
    print(f'the threshold based jitter frame exclusion gives: {np.shape(indices_of_highest_drift)}')
    
    # creates a bridged version of the overly jittery frames (to contend with single frame outliers)
    video_indices = add_intermediate_elements(video_indices, gap_to_bridge=gap_to_bridge)
    # This is the input you should give to the BlockSync.remove_eye_datapoints function (which already maps it to the df) 
    
    
    # translates the video indices to le/re dataframe rows
    df_indices_to_remove = df_dict[eye].loc[df_dict[eye][eye_frame_col].isin(video_indices)].index.values
    
    return df_indices_to_remove, video_indices

def bokeh_plotter(data_list, label_list,
                  plot_name='default',
                  x_axis='X', y_axis='Y',
                  peaks=None, peaks_list=False, export_path=False):
    """Generates an interactive Bokeh plot for the given data vector.
    Args:
        data_list (list or array): The data to be plotted.
        label_list (list of str): The labels of the data vectors
        plot_name (str, optional): The title of the plot. Defaults to 'default'.
        x_axis (str, optional): The label for the x-axis. Defaults to 'X'.
        y_axis (str, optional): The label for the y-axis. Defaults to 'Y'.
        peaks (list or array, optional): Indices of peaks to highlight on the plot. Defaults to None.
        export_path (False or str): when set to str, will output the resulting html fig
    """
    color_cycle = cycle(bokeh.palettes.Category10_10)
    fig = bokeh.plotting.figure(title=f'bokeh explorer: {plot_name}',
                                x_axis_label=x_axis,
                                y_axis_label=y_axis,
                                plot_width=1500,
                                plot_height=700)

    for i, vec in enumerate(range(len(data_list))):
        color = next(color_cycle)
        data_vector = data_list[vec]
        if label_list is None:
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"Line {len(fig.renderers)}")
        elif len(label_list) == len(data_list):
            fig.line(range(len(data_vector)), data_vector, line_color=color, legend_label=f"{label_list[i]}")
        if peaks is not None and peaks_list is True:
            fig.circle(peaks[i], data_vector[peaks[i]], size=10, color=color)

    if peaks is not None and peaks_list is False:
        fig.circle(peaks, data_vector[peaks], size=10, color='red')

    if export_path is not False:
        print(f'exporting to {export_path}')
        bokeh.io.output.output_file(filename=str(export_path / f'{plot_name}.html'), title=f'{plot_name}')
    bokeh.plotting.show(fig)
    
def play_video_with_ellipses_rotation(block, eye, path_to_video=False, xflip=False, transformation_matrix=None, phi_in_radians=False):
    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.left_eye_data
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.right_eye_data
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    
    if video_path is not False:
        video_path = path_to_video

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Apply transformation matrix if provided
        if transformation_matrix is not None:
            frame = cv2.warpAffine(frame, transformation_matrix, (frame.shape[1], frame.shape[0]))

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue

        # Extract ellipse parameters
        if transformation_matrix is not None:
            try:
                center_x = int(current_frame_data['center_x'])
                center_y = int(current_frame_data['center_y'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                if phi_in_radians:
                    phi = np.deg2rad(float(current_frame_data['phi']))
                else:
                    phi = float(current_frame_data['phi'])
                
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue
        else:
            try:
                center_x = int(current_frame_data['center_x'])
                center_y = int(current_frame_data['center_y'])
                width = int(current_frame_data['width'])
                height = int(current_frame_data['height'])
                if phi_in_radians:
                    phi = np.deg2rad(float(current_frame_data['phi']))
                else:
                    phi = float(current_frame_data['phi'])
        
                # Draw the ellipse on the frame
                cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
                
                # Add text to the frame
                text = f'ellipse angle: {phi}'
                cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # Display the frame
                cv2.imshow('Video with Ellipses', frame)
            
                # Check for the 'q' key to quit
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            except ValueError:
                continue

    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()
    
def play_video_with_ellipses_rotation_plus_major_axis(block, eye, path_to_video=False, xflip=False, transformation_matrix=None):
    if eye == 'left':
        video_path = block.le_videos[0]
        ellipse_dataframe = block.left_eye_data
    elif eye == 'right':
        video_path = block.re_videos[0]
        ellipse_dataframe = block.right_eye_data
    else:
        raise ValueError(f"eye can only be 'left' or 'right'")
    
    if video_path is not False:
        video_path = path_to_video

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error opening video file.")
        return

    # Loop through each frame
    while True:
        # Read a frame from the video
        ret, frame = cap.read()
    
        if not ret:
            # Break the loop if the video is finished
            break
        
        # Optionally flip the frame along the x-axis
        if xflip:
            frame = cv2.flip(frame, 1)

        # Apply transformation matrix if provided
        if transformation_matrix is not None:
            frame = cv2.warpAffine(frame, transformation_matrix, (frame.shape[1], frame.shape[0]))

        # Get the corresponding ellipse data for the current frame
        current_frame_num = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
        try:
            current_frame_data = ellipse_dataframe.iloc[ellipse_dataframe.query('eye_frame == @current_frame_num').index[0]]
        except IndexError:
            continue

        # Extract ellipse parameters
        try:
            center_x = int(current_frame_data['center_x'])
            center_y = int(current_frame_data['center_y'])
            width = int(current_frame_data['major_ax'])
            height = int(current_frame_data['minor_ax'])
            phi = np.deg2rad(float(current_frame_data['phi']))  # Convert angle to radians
            
            # Draw the ellipse on the frame
            cv2.ellipse(frame, (center_x, center_y), (width, height), phi, 0, 360, (0, 255, 0), 2)
            
            # Calculate endpoints of major axis
            axis_length = max(width, height) / 2
            sin_phi = np.sin(phi)
            cos_phi = np.cos(phi)
            x1 = int(center_x + axis_length * cos_phi)
            y1 = int(center_y + axis_length * sin_phi)
            x2 = int(center_x - axis_length * cos_phi)
            y2 = int(center_y - axis_length * sin_phi)
            
            # Draw major axis
            cv2.line(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
            
            # Add text to the frame
            text = f'ellipse angle: {np.rad2deg(phi)}'
            cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Display the frame
            cv2.imshow('Video with Ellipses', frame)
        
            # Check for the 'q' key to quit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        except ValueError:
            continue

    # Release video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()
    
def get_frame_count(video_path):
        """
        Get the number of frames for the video in the specified path using OpenCV.
    
        Parameters:
            video_path (str): Path to the video file.
    
        Returns:
            int: Number of frames in the video.
        """
        
        # Open the video file
        cap = cv2.VideoCapture(video_path)
    
        # Check if the video file is opened successfully
        if not cap.isOpened():
            print("Error: Could not open the video file.")
            return -1
    
        # Get the total number of frames in the video
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
        # Release the VideoCapture object
        cap.release()
    
        return frame_count


# Block Instantiation

In [361]:

# define a single block to synchronize and finally export l/r_eye_data csv files:
# this step creates block_collection - a list of BlockSync objects of interest
block_numbers = [26]
bad_blocks = [] # 
experiment_path = pathlib.Path(r"Z:\Nimrod\experiments")
animal = 'PV_62'

block_collection = uf.block_generator(block_numbers=block_numbers,
                                      experiment_path=experiment_path,
                                      animal=animal,
                                      bad_blocks=bad_blocks,regev=True)
# create a block_dict object for ease of access:
block_dict = {}
for b in block_collection:
    block_dict[str(b.block_num)] = b
block = block_collection[0]

instantiated block number 026 at Path: Z:\Nimrod\experiments\PV_62\2023_04_27\block_026, new OE version
Found the sample rate for block 026 in the xml file, it is 20000 Hz
created the .oe_rec attribute as an open ephys recording obj with get_data functionality
retrieving zertoh sample number for block 026
got it!


In [5]:
from scipy.interpolate import interp1d
import numpy as np
import pandas as pd
import pathlib
from tqdm import tqdm

def synchronize_block_for_non_60fps_acquisition(self, export=True, overwrite=False, target_frame_rate=60, margin_of_error=0.1):
    """
    Synchronize the video frames to a target frame rate using interpolation.
    """
    # Check if previously exported file exists
    if pathlib.Path(self.analysis_path / 'blocksync_df.csv').exists() and not overwrite:
        self.blocksync_df = pd.read_csv(pathlib.Path(self.analysis_path / 'blocksync_df.csv'), engine='python')
        print('blocksync_df loaded from analysis folder')
        return self.blocksync_df
    
    print('Creating blocksync_df')

    # Define start and end times
    start_time = max([self.arena_vid_first_t, self.r_vid_first_t, self.l_vid_first_t])
    end_time = min([self.arena_vid_last_t, self.r_vid_last_t, self.l_vid_last_t])

    # Extract TTLs and calculate frame rate
    arena_ttls = self.oe_events.query('@start_time < Arena_TTL < @end_time')['Arena_TTL'].values
    arena_frame_rate = self.sample_rate / np.median(np.diff(arena_ttls))

    if not (target_frame_rate - margin_of_error <= arena_frame_rate <= target_frame_rate + margin_of_error):
        print(f"Arena video frame rate is {arena_frame_rate:.2f} Hz. Adjusting to {target_frame_rate} FPS.")

        # Define target time base
        original_time = np.cumsum(np.insert(np.diff(arena_ttls), 0, 0)) / self.sample_rate
        target_time = np.arange(0, original_time[-1], 1 / target_frame_rate)

        # Interpolate using linear method
        interpolator = interp1d(original_time, arena_ttls, kind='linear', fill_value='extrapolate')
        new_arena_ttl = interpolator(target_time).astype(int)
    else:
        print(f"Arena video frame rate is {arena_frame_rate:.2f} Hz, within acceptable range. No adjustment needed.")
        new_arena_ttl = arena_ttls

    # Create a synchronization DataFrame
    arena_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time')[['Arena_TTL', 'Arena_TTL_frame']]
    r_eye_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time or Arena_TTL != Arena_TTL')[['R_eye_TTL', 'R_eye_TTL_frame']].dropna()
    l_eye_tf = self.oe_events.query('@start_time < Arena_TTL < @end_time or Arena_TTL != Arena_TTL')[['L_eye_TTL', 'L_eye_TTL_frame']].dropna()

    # Build the synchronization DataFrame
    self.blocksync_df = pd.DataFrame(columns=['Arena_frame', 'L_eye_frame', 'R_eye_frame'], index=new_arena_ttl)

    for t in tqdm(new_arena_ttl, desc='Synchronizing Frames'):
        arena_frame = arena_tf['Arena_TTL_frame'].iloc[self.get_closest_frame(t, arena_tf['Arena_TTL'])]
        l_eye_frame = l_eye_tf['L_eye_TTL_frame'].iloc[self.get_closest_frame(t, l_eye_tf['L_eye_TTL'])]
        r_eye_frame = r_eye_tf['R_eye_TTL_frame'].iloc[self.get_closest_frame(t, r_eye_tf['R_eye_TTL'])]
        self.blocksync_df.loc[t] = [arena_frame, l_eye_frame, r_eye_frame]

    print('Created blocksync_df')
    if export:
        self.blocksync_df.to_csv(self.analysis_path / 'blocksync_df.csv')
        print(f'Exported blocksync_df to {self.analysis_path}/blocksync_df.csv')

    return self.blocksync_df


In [10]:
import pickle
import cv2
from pathlib import Path


def select_rois_for_blocks(block_list, save_path):
    """
    Allows the user to select ROIs for all blocks without calculating brightness.

    Args:
        block_list (list): List of BlockSync objects.
        save_path (str or Path): Path to save the selected ROIs as a pickle file.

    Returns:
        None
    """
    rois_dict = {}
    for block in block_list:
        rois_dict[block.block_num] = {}

        for eye, video_path in zip(['left_eye', 'right_eye'], [block.le_videos[0], block.re_videos[0]]):
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Cannot open video {video_path}")
                continue

            ret, frame = cap.read()
            if not ret:
                print(f"Error: Cannot read the first frame of {video_path}")
                cap.release()
                continue

            roi = cv2.selectROI(f"Select ROI for {eye} - Block {block.block_num}", frame, showCrosshair=True, fromCenter=False)
            rois_dict[block.block_num][eye] = roi
            cv2.destroyWindow(f"Select ROI for {eye} - Block {block.block_num}")
            cap.release()

    # Save the ROIs to a pickle file
    save_path = Path(save_path)
    with open(save_path, 'wb') as f:
        pickle.dump(rois_dict, f)

    print(f"ROIs saved to {save_path}")


def calculate_brightness_for_blocks(block_list, rois_path, threshold_value=30, export=True):
    """
    Calculates brightness vectors for all blocks using preselected ROIs.

    Args:
        block_list (list): List of BlockSync objects.
        rois_path (str or Path): Path to the pickle file containing saved ROIs.
        threshold_value (float): Threshold value to use as a mask before calculating brightness.
        export (bool): If True, exports the brightness vectors into a pickle file.

    Returns:
        None
    """
    rois_path = Path(rois_path)
    if not rois_path.exists():
        raise FileNotFoundError(f"ROIs file {rois_path} does not exist.")

    # Load the saved ROIs
    with open(rois_path, 'rb') as f:
        rois_dict = pickle.load(f)

    for block in block_list:
        if block.block_num not in rois_dict:
            print(f"Skipping Block {block.block_num}: No ROI data found.")
            continue

        block_rois = rois_dict[block.block_num]

        print(f"Calculating brightness for Block {block.block_num}...")

        # Calculate brightness for left eye
        block.le_frame_val_list = block.produce_frame_val_list_with_roi(
            block.le_videos[0], block_rois['left_eye'], threshold_value
        )

        # Calculate brightness for right eye
        block.re_frame_val_list = block.produce_frame_val_list_with_roi(
            block.re_videos[0], block_rois['right_eye'], threshold_value
        )

        if export:
            export_path = block.analysis_path / 'eye_brightness_values_dict.pkl'
            frame_val_dict = {
                'left_eye': block.le_frame_val_list,
                'right_eye': block.re_frame_val_list
            }
            with open(export_path, 'wb') as f:
                pickle.dump(frame_val_dict, f)
            print(f"Brightness vectors saved to {export_path}")

        print(f"Brightness calculation complete for Block {block.block_num}.")


In [13]:
save_path = r"Z:\Nimrod\experiments\PV_57\helper_files/rois.pkl"  # Path to save the selected ROIs
select_rois_for_blocks(block_collection, save_path)

ROIs saved to Z:\Nimrod\experiments\PV_57\helper_files\rois.pkl


In [15]:
calculate_brightness_for_blocks(block_collection,save_path)

Processing Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\LE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4:   0%|          | 160/54465 [00:00<00:33, 1599.95frame/s]

Calculating brightness for Block 001...
Working on video Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\LE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4


Processing Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\LE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4: 100%|██████████| 54465/54465 [00:28<00:00, 1909.40frame/s]
Processing Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\RE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4:   0%|          | 174/54492 [00:00<00:31, 1724.76frame/s]

Finished video Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\LE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4, processed 54465 frames
Working on video Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\RE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4


Processing Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\eye_videos\RE\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\pv_57_single_print_01.mp4: 100%|██████████| 54492/54492 [00:32<00:00, 1670.02frame/s]


...

Brightness vectors saved to Z:\Nimrod\experiments\PV_57\2024_12_01\block_012\analysis\eye_brightness_values_dict.pkl
Brightness calculation complete for Block 012.
Calculating brightness for Block 013...
Working on video Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\LE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4


Processing Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\LE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4: 100%|██████████| 49836/49836 [00:25<00:00, 1928.60frame/s]
Processing Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\RE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4:   0%|          | 168/49845 [00:00<00:29, 1663.40frame/s]

Finished video Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\LE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4, processed 49836 frames
Working on video Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\RE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4


Processing Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\RE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4: 100%|██████████| 49845/49845 [00:25<00:00, 1949.28frame/s]


Finished video Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\RE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4, processed 49845 frames
Brightness vectors saved to Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis\eye_brightness_values_dict.pkl
Brightness calculation complete for Block 013.


In [12]:
for block in block_collection:
    block.handle_eye_videos()

handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.h264', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\RE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\RE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\RE\\pv_57_single_print_0

In [6]:
# This step is used to quickly go over the analyzed blocks and load their internal data
for block in block_collection:
    block.handle_arena_files()
    block.parse_open_ephys_events()
    block.handle_eye_videos()
    block.get_eye_brightness_vectors()
    #block.synchronize_block()
    block.synchronize_block_for_non_60fps_acquisition(export=True,
                                                      overwrite=True,
                                                      target_frame_rate=60,
                                                      margin_of_error=5)
    block.create_eye_brightness_df(threshold_value=20)
    
    # if the code fails here, go to manual synchronization
#    block.import_manual_sync_df()

handling arena files
Arena video Names:
front_20241121T153325.mp4
left_20241121T153325.mp4
right_20241121T153325.mp4
top_20241121T153325.mp4
running parse_open_ephys_events...
block 001 has a parsed events file, reading...
handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.h264', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\RE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_001\\eye_videos\\LE\\pv_57_single_print_01_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_01_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_5

  1%|          | 353/53581 [00:00<00:15, 3529.99it/s]

creating blocksync_df
Arena video frame rate is 59.17 Hz, within acceptable range. No adjustment needed.


100%|██████████| 53581/53581 [00:13<00:00, 3997.64it/s]


created blocksync_df
exported blocksync_df to Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\analysis/ blocksync_df.csv
creating Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\analysis/eye_brightness_df.csv
handling arena files
Arena video Names:
front_20241121T155541.mp4
left_20241121T155541.mp4
right_20241121T155541.mp4
top_20241121T155541.mp4
running parse_open_ephys_events...
block 002 has a parsed events file, reading...
handling eye video files
converting videos...
converting files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_002\\eye_videos\\LE\\pv_57_single_print_02_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_02.h264', 'Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_002\\eye_videos\\RE\\pv_57_single_print_02_640x480_60hz_experiment_1_recording_0\\pv_57_single_print_02.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_11_21\\block_002\\eye_videos\\LE\\pv_57_single_print_02_640x480_60hz_experiment_1_recording_0\\pv_57_singl

...

created blocksync_df
exported blocksync_df to Z:\Nimrod\experiments\PV_57\2024_12_01\block_012\analysis/ blocksync_df.csv
creating Z:\Nimrod\experiments\PV_57\2024_12_01\block_012\analysis/eye_brightness_df.csv
handling arena files
Arena video Names:
front_20241201T163514.mp4
left_20241201T163514.mp4
right_20241201T163514.mp4
top_20241201T163514.mp4
running parse_open_ephys_events...
block 013 has a parsed events file, reading...
handling eye video files
converting videos...


  0%|          | 0/53581 [00:00<?, ?it/s]

converting files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\LE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04.h264', 'Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\RE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04.h264'] 
 avoiding conversion on files: ['Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\LE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\LE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04_LE.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\RE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04.mp4', 'Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\eye_videos\\RE\\d3__04_640x480_60hz_experiment_1_recording_0\\d3__04_RE.mp4']
The file Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\LE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4 already exists, no convers

100%|██████████| 53581/53581 [00:13<00:00, 3841.99it/s]


created blocksync_df
exported blocksync_df to Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis/ blocksync_df.csv
creating Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\analysis/eye_brightness_df.csv


In [123]:
for block in block_collection:
    block.read_dlc_data(overwrite=True, export=True)
    #block.calibrate_pixel_size(10)

c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████| 54463/54463 [00:33<00:00, 1631.26it/s]



 ellipses calculation complete


100%|██████████| 54490/54490 [00:33<00:00, 1613.27it/s]
C:\Users\MarkS3\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\reshape\merge.py:1112: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():



 ellipses calculation complete
created le / re dataframes
exporting to analysis folder


c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████| 75418/75418 [00:46<00:00, 1618.13it/s]


...


 ellipses calculation complete
created le / re dataframes
exporting to analysis folder


c:\users\marks3\pycharmprojects\pythonproject3\venv\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
100%|██████████| 49834/49834 [00:28<00:00, 1720.48it/s]



 ellipses calculation complete


100%|██████████| 49843/49843 [00:30<00:00, 1611.64it/s]
C:\Users\MarkS3\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\reshape\merge.py:1112: RuntimeWarning: invalid value encountered in cast
  if not (lk == lk.astype(rk.dtype))[~np.isnan(lk)].all():



 ellipses calculation complete
created le / re dataframes
exporting to analysis folder


In [124]:
for block in block_collection:
    block.get_jitter_reports(export=True, overwrite=False, remove_led_blinks=False, sort_on_loading=True)

jitter report loaded from analysis folder
results saved to Z:\Nimrod\experiments\PV_57\2024_11_21\block_001\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_dict attributes
jitter report loaded from analysis folder
results saved to Z:\Nimrod\experiments\PV_57\2024_11_21\block_002\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_dict attributes
jitter report loaded from analysis folder
results saved to Z:\Nimrod\experiments\PV_57\2024_11_21\block_003\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_dict attributes
jitter report loaded from analysis folder
results saved to Z:\Nimrod\experiments\PV_57\2024_11_21\block_004\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_dict attributes
jitter report loaded from analysis folder
results saved to Z:\Nimrod\experiments\PV_57\2024_11_25\block_005\analysis\jitter_report_dict.pkl
Got the jitter report - check out re/le_jitter_d

In [125]:
for block in block_collection:
#    block.get_jitter_reports(export=True, overwrite=True, remove_led_blinks=False, sort_on_loading=True)
    block.correct_jitter()
    block.find_led_blink_frames(plot=True)
    block.remove_led_blinks_from_eye_df(export=True)

100%|██████████| 36/36 [00:00<00:00, 3999.65it/s]

The right eye std of the X coord was 17.068908727991413
After correction it is: 17.085446672029057
The right eye std of the Y coord was 17.115669246049826
After correction it is: 17.036504142955067

 The left eye std of the X coord was 15.090810175602341
After correction it is: 15.105009208881155

 The left eye std of the Y coord was 17.659165602271962
After correction it is: 17.982080501311415
hi new version
collecting left-eye data
data length is 54465
z_score length is 54465



100%|██████████| 36/36 [00:00<00:00, 3999.34it/s]

collecting right eye data
data length is 54492
z_score length is 54492


removed led blink data from le / re dataframes
exported nan filled dataframes to csv
The right eye std of the X coord was 17.946259759076572
After correction it is: 28.259195025411277
The right eye std of the Y coord was 17.83160036358416
After correction it is: 24.896536067397715


...

removed led blink data from le / re dataframes
exported nan filled dataframes to csv
The right eye std of the X coord was 13.249581771259011
After correction it is: 13.342795792277025
The right eye std of the Y coord was 12.173645348222955
After correction it is: 11.977040484162751

 The left eye std of the X coord was 15.053855275361395
After correction it is: 15.827086857626933

 The left eye std of the Y coord was 18.22991442595649
After correction it is: 17.099026700893713


  0%|          | 0/33 [00:00<?, ?it/s]

hi new version
collecting left-eye data
data length is 49836
z_score length is 49836
collecting right eye data
data length is 49845


100%|██████████| 33/33 [00:00<00:00, 3666.45it/s]


z_score length is 49845
removed led blink data from le / re dataframes
exported nan filled dataframes to csv


In [126]:
block_dict.keys()

dict_keys(['001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013'])

In [344]:
block=block_dict['013']

In [345]:
df_inds_to_remove_l, vid_inds_l = find_jittery_frames(block,'left',max_distance=60, diff_threshold=5, gap_to_bridge=24)
df_inds_to_remove_r, vid_inds_r = find_jittery_frames(block,'right',max_distance=60, diff_threshold=5, gap_to_bridge=24)

# These are verification plots for the jitter outlier removal functions:
# to verify, I want a bokeh explorable:
rdf = pd.DataFrame.from_dict(block.re_jitter_dict)
ldf = pd.DataFrame.from_dict(block.le_jitter_dict)

the diff based jitter frame exclusion gives: (259,)
the threshold based jitter frame exclusion gives: (365,)
the diff based jitter frame exclusion gives: (65,)
the threshold based jitter frame exclusion gives: (15,)


In [346]:
bokeh_plotter([rdf.top_correlation_dist], ['drift_distance'], peaks=vid_inds_r)

In [347]:
bokeh_plotter([ldf.top_correlation_dist], ['drift_distance'], peaks=vid_inds_l)

In [348]:
block.remove_eye_datapoints_based_on_video_frames('right', indices_to_nan=vid_inds_r)
block.remove_eye_datapoints_based_on_video_frames('left', indices_to_nan=vid_inds_l)

removed 228 from the right eye dataframe
removed 1148 from the left eye dataframe


# Data rotation

In [349]:
block.rotate_data_according_to_frame_ref('left')

Please select two points on the frame.
left rotation matrix: 
 [[   0.71137921   -0.70280838  261.0326629 ]
 [   0.70280838    0.71137921 -155.62969195]] 
 left rotation angle: 
 -44.652757102914244
left data rotated


In [350]:
block.rotate_data_according_to_frame_ref('right')

Please select two points on the frame.
right rotation matrix: 
 [[  0.75288548   0.65815154 -78.87972396]
 [ -0.65815154   0.75288548 269.9159793 ]] 
 right rotation angle: 
 41.159050974663046
right data rotated


In [351]:
block.create_eye_data()

Index(['Arena_TTL', 'R_eye_frame', 'ms_axis', 'center_x_rotated',
       'center_y_rotated', 'phi_rotated', 'width', 'height', 'major_ax',
       'minor_ax', 'ratio'],
      dtype='object')
   OE_timestamp  eye_frame   ms_axis    center_x    center_y        phi  \
0     1482496.0       18.0  74124.80  252.722856  322.255968  31.410567   
1     1482834.0       19.0  74141.70  253.217326  322.139737  32.446504   
2     1483173.0       20.0  74158.65  253.491721  322.237446  32.691181   
3     1483511.0       21.0  74175.55  253.479622  322.097218  32.495051   
4     1483849.0       22.0  74192.45  253.444468  322.394839  32.593095   

       width     height   major_ax   minor_ax     ratio  
0  39.862326  26.029617  39.862326  26.029617  1.531422  
1  40.005766  26.032208  40.005766  26.032208  1.536780  
2  39.569541  26.134329  39.569541  26.134329  1.514083  
3  39.565902  26.113054  39.565902  26.113054  1.515177  
4  39.629568  25.953247  39.629568  25.953247  1.526960  
Index(['Are

C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:2786: RuntimeWarning: All-NaN axis encountered
  df['major_ax'] = np.nanmax(df[['width', 'height']], axis=1)
C:\Users\MarkS3\PycharmProjects\pythonProject3\BlockSync_current.py:2787: RuntimeWarning: All-NaN axis encountered
  df['minor_ax'] = np.nanmin(df[['width', 'height']], axis=1)


In [352]:
# This bit examine the ellipses to verify phi jump issues
block.right_eye_data['phi'] = block.right_eye_data['phi'] + 90

In [353]:
block.left_eye_data['phi'] = block.left_eye_data['phi'] + 90

# Rotation eye data Verification

In [354]:
# right eye inspection after rotation
#path_to_video = [x for x in pathlib.Path(block.re_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
path_to_video = [x for x in pathlib.Path(block.re_videos[0]).parent.iterdir() if '.mp4' in str(x.name)][0]
print(path_to_video)
play_video_with_ellipses_rotation(block=block ,eye='right', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=block.right_rotation_matrix)

Z:\Nimrod\experiments\PV_57\2024_12_01\block_013\eye_videos\RE\d3__04_640x480_60hz_experiment_1_recording_0\d3__04.mp4


In [355]:
# left eye inspection
#path_to_video = [x for x in pathlib.Path(block.le_videos[0]).parent.iterdir() if '.mp4' in str(x.name) and 'DLC' in str(x.name)][0]
path_to_video = [x for x in pathlib.Path(block.le_videos[0]).parent.iterdir() if '.mp4' in str(x.name)][0]
play_video_with_ellipses_rotation(block=block,eye='left', path_to_video=str(path_to_video) ,xflip=True, transformation_matrix=block.left_rotation_matrix)

# Eye videos relative lag correction

In [369]:
# block integrated version:
block.find_led_blink_frames()
l_df, r_df = block.correct_relative_eye_drift_based_on_LED_lights_out(verification_plots=False)
block.left_eye_data = l_df
block.right_eye_data = r_df

100%|██████████| 71/71 [00:00<00:00, 8404.23it/s]

collecting left-eye data
data length is 107642
z_score length is 107642
collecting right eye data
data length is 107640
z_score length is 107640
missing frame at 7
missing frame at 11
missing frame at 27
missing frame at 28
missing frame at 29
99208 99208
0
0
0 S
1 S
2 S
3 S
4 S
5 S
6 S
7 S
8 S
9 S
10 S
11 S
12 S
13 S
14 S
15 S
16 S
17 S
18 S
19 S
20 S
21 S
22 S
23 S
24 S


In [368]:
l_df, r_df = block.correct_eye_sync_based_on_OE_LED_events()

missing frame at 7
missing frame at 11
double row at 17
double row at 23
missing frame at 27
missing frame at 28
missing frame at 29


KeyError: "None of [Index(['LED_driver'], dtype='object')] are in the [columns]"

In [370]:
#finalize here, then export
block.left_eye_data = l_df
block.right_eye_data = r_df

# End of Synchronization Pipeline

# Data export steps:

In [360]:
def export_eye_data_2d_w_rotation_matrix(block):
    """
    This function saves the eye dataframes to two csv files
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    block.right_eye_data.to_csv(block.analysis_path / 'right_eye_data.csv')
    block.left_eye_data.to_csv(block.analysis_path / 'left_eye_data.csv')
    rotation_dict = {'left_rotation_matrix': block.left_rotation_matrix,
                'left_rotation_angle':  block.left_rotation_angle,
                'right_rotation_matrix':block.right_rotation_matrix,
                'right_rotation_angle': block.right_rotation_angle}
    with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'wb') as file:
        pickle.dump(rotation_dict, file)
        print(f'eye dataframes and rotation matrix saved to: {file}')
        
export_eye_data_2d_w_rotation_matrix(block)

eye dataframes and rotation matrix saved to: <_io.BufferedWriter name='Z:\\Nimrod\\experiments\\PV_57\\2024_12_01\\block_013\\analysis\\rotate_eye_data_params.pkl'>


In [64]:
block.final_sync_df

# Manual Synchronization steps - only use when needed and after block instantiation

In [389]:
block_dict.keys()

dict_keys(['026'])

In [114]:
block = block_dict['013']

In [115]:
block.get_eyes_diff_list(2)

index error on position 39 out of 40
index error on position 59 out of 60
The suspected lag between eye cameras is 15.0 with the direction ['right', 'late']


In [116]:
block.fix_eye_synchronization()

created manual_sync_df attribute for the block


In [371]:
block.manual_sync_df

,Unnamed: 0,Unnamed: 0.1,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,0,0,565951.0,913.0,1.0,NaN,-2.646521,NaN
1,1,1,566291.0,914.0,2.0,NaN,-2.646440,NaN
2,2,2,566631.0,915.0,3.0,NaN,-2.646465,NaN
3,3,3,566970.0,916.0,4.0,NaN,-2.641369,NaN
4,4,4,567310.0,917.0,5.0,2.0,-2.641731,0.554543
...,...,...,...,...,...,...,...,...
99203,99203,99203,34261949.0,100116.0,101272.0,101271.0,0.072080,-1.185376
99204,99204,99204,34262289.0,100117.0,101273.0,101272.0,0.072150,-1.185574
99205,99205,99205,34262628.0,100118.0,101274.0,101273.0,0.072569,-1.186444
99206,99206,99206,34262968.0,100119.0,101275.0,101274.0,0.075081,-1.186171


In [119]:
# use this to manually shift L\R eye synchronization
eye_to_move = 'R'
block.move_eye_sync_manual(cols_to_move=[f'{eye_to_move}_eye_frame',f'{eye_to_move}_values'],step=-2)

In [120]:
block.full_sync_verification(ms_axis=False,with_arena=False)

In [121]:
block.export_manual_sync_df()
block.import_manual_sync_df()

In [372]:
def full_sync_verification(self):
    from bokeh.plotting import figure, show, curdoc
    from bokeh.models import Slider, ColumnDataSource, CustomJS
    from bokeh.layouts import column
    import numpy as np
    data_dict = {
    'L_values': [None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['L_values']],
    'R_values': [None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['R_values']]
    }
    source = ColumnDataSource(data=dict(
    x_axis=self.manual_sync_df.index / self.sample_rate,
    left_y=[None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['L_values'].shift(0)],
    right_y=[None if np.isnan(x) or np.isinf(x) else x for x in
    self.manual_sync_df['R_values'].shift(0)]
    ))
    bokeh_fig = figure(
    title=f'Full Synchronization Verification',
    x_axis_label='Seconds',
    y_axis_label='Brightness Z_Score',
    plot_width=1500,
    plot_height=700
    )
    left_line = bokeh_fig.line('x_axis', 'left_y', source=source,
    legend_label='Left_eye_values', line_width=1,
    line_color='blue')
    right_line = bokeh_fig.line('x_axis', 'right_y', source=source,
    legend_label='Right_eye_values', line_width=1,
    line_color='red')
    slider_left = Slider(start=-50, end=50, value=0, step=1, title="Left Eye Shift")
    slider_right = Slider(start=-50, end=50, value=0, step=1, title="Right Eye Shift")
    callback = CustomJS(args=dict(source=source, slider_left=slider_left,
    slider_right=slider_right, data_dict=data_dict),
    code="""
    const plot_data = source.data;
    const left_shift = slider_left.value;
    const right_shift = slider_right.value;
    const left_values = data_dict['L_values'];
    const right_values = data_dict['R_values'];
    const length = plot_data['x_axis'].length;
    for (let i = 0; i < length; i++) {
    plot_data['left_y'][i] = (i + left_shift >= 0 && i +
    left_shift < length) ? left_values[i + left_shift] : null;
    plot_data['right_y'][i] = (i + right_shift >= 0 && i +
    right_shift < length) ? right_values[i + right_shift] : null;
    }
    console.log("Left shift: ", left_shift, "Right shift: ",
    right_shift); // Debug output
    source.change.emit();
    """)
    slider_left.js_on_change('value', callback)
    slider_right.js_on_change('value', callback)
    layout = column(bokeh_fig, slider_left, slider_right)
    curdoc().add_root(layout)
    show(layout)


In [373]:
full_sync_verification(block)

In [38]:
block.manual_sync_df.rename(columns={'Unnamed: 0.1': 'Arena_TTL'},inplace=True)

In [39]:
block.manual_sync_df

,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,913655,845.0,6.0,2.0,-0.970385,-0.877140
1,913988,846.0,6.0,3.0,-0.970385,-0.872642
2,914321,846.0,8.0,4.0,-0.969849,-0.871553
3,914655,847.0,9.0,5.0,-0.966254,-0.868974
4,914988,847.0,10.0,6.0,-0.970299,-0.867236
...,...,...,...,...,...,...
121570,41436988,62567.0,121799.0,121794.0,-0.277042,0.023131
121571,41437321,62567.0,121799.0,NaN,-0.277042,NaN
121572,41437655,62568.0,121800.0,NaN,-0.285377,NaN
121573,41437988,62568.0,121802.0,NaN,-0.269196,NaN


In [167]:
block.left_eye_data

,OE_timestamp,eye_frame,ms_axis,center_x,center_y,phi,width,height,major_ax,minor_ax,ratio
0,676740.0,5.0,33837.00,319.518120,177.992945,-15.443761,46.028471,40.458124,46.028471,40.458124,1.137682
1,677073.0,6.0,33853.65,319.537921,177.984564,-15.536813,46.187965,40.480872,46.187965,40.480872,1.140982
2,677397.0,7.0,33869.85,319.510882,178.011487,-15.462979,46.079456,40.440812,46.079456,40.440812,1.139430
3,677730.0,8.0,33886.50,319.491438,178.003452,-14.895568,45.985501,40.541131,45.985501,40.541131,1.134293
4,678053.0,9.0,33902.65,319.491147,178.091727,-14.170065,45.875067,40.475895,45.875067,40.475895,1.133392
...,...,...,...,...,...,...,...,...,...,...,...
97994,32834371.0,96856.0,1641718.55,340.761068,109.354315,-24.174997,41.280409,28.235806,41.280409,28.235806,1.461988
97995,32834704.0,96857.0,1641735.20,340.931360,109.326126,-24.160139,41.655405,28.352723,41.655405,28.352723,1.469185
97996,32835027.0,96858.0,1641751.35,340.753870,109.435444,-24.996913,42.075840,28.446548,42.075840,28.446548,1.479119
97997,32835360.0,96859.0,1641768.00,341.181379,109.345112,-23.876929,41.828722,28.359923,41.828722,28.359923,1.474924


In [ ]:
# This bit of code goes over blocks and collects the median distance between the rostral and caudal edges
import os
from ellipse import LsqEllipse
import scipy.stats as stats

def eye_tracking_analysis(dlc_video_analysis_csv, uncertainty_thr):
    """
    :param dlc_video_analysis_csv: the csv output of a dlc analysis of one video, already read by pandas with header=1
    :param uncertainty_thr: The confidence P value to use as a threshold for datapoint validity in the analysis
    :returns ellipse_df: a DataFrame of ellipses parameters (center, width, height, phi, size) for each video frame

    """
    # import the dataframe and convert it to floats
    data = dlc_video_analysis_csv
    data = data.iloc[1:].apply(pd.to_numeric)

    # sort the pupil elements to dfs: x and y, with p as probability
    pupil_elements = np.array([x for x in data.columns if 'Pupil' in x])

    # get X coords
    pupil_xs_before_flip = data[pupil_elements[np.arange(0, len(pupil_elements), 3)]]

    # flip the data around the midpoint of the x-axis (shooting the eye through a camera flips right and left)
    pupil_xs = 320 * 2 - pupil_xs_before_flip

    # get Y coords (no need to flip as opencv conventions already start with origin at top left of frame
    # and so, positive Y is maintained as up in a flipped image as we have)
    pupil_ys = data[pupil_elements[np.arange(1, len(pupil_elements), 3)]]
    pupil_ps = data[pupil_elements[np.arange(2, len(pupil_elements), 3)]]

    # rename dataframes for masking with p values of bad points:
    pupil_ps = pupil_ps.rename(columns=dict(zip(pupil_ps.columns, pupil_xs.columns)))
    pupil_ys = pupil_ys.rename(columns=dict(zip(pupil_ys.columns, pupil_xs.columns)))
    good_points = pupil_ps > uncertainty_thr
    pupil_xs = pupil_xs[good_points]
    pupil_ys = pupil_ys[good_points]

    # Do the same for the edges
    edge_elements = np.array([x for x in data.columns if 'edge' in x])
    edge_xs_before_flip = data[edge_elements[np.arange(0, len(edge_elements), 3)]]
    edge_xs = 320*2 - edge_xs_before_flip
    edge_ys = data[edge_elements[np.arange(1, len(edge_elements), 3)]]
    edge_ps = data[edge_elements[np.arange(2,len(edge_elements),3)]]
    edge_ps = edge_ps.rename(columns=dict(zip(edge_ps.columns,edge_xs.columns)))
    edge_ys = edge_ys.rename(columns=dict(zip(edge_ys.columns,edge_xs.columns)))
    good_edge_points = edge_ps < uncertainty_thr
    
    # work row by row to figure out the ellipses
    ellipses = []
    caudal_edge_ls = []
    rostral_edge_ls = []
    for row in tqdm.tqdm(range(1, len(data) - 1)):
        # first, take all the values, and concatenate them into an X array
        x_values = pupil_xs.loc[row].values
        y_values = pupil_ys.loc[row].values
        X = np.c_[x_values, y_values]

        # now, remove nan values, and check if there are enough points to make the ellipse
        X = X[~ np.isnan(X).any(axis=1)]

        # if there are enough rows for a fit, make an ellipse
        if X.shape[0] > 5:
            el = LsqEllipse().fit(X)
            center, width, height, phi = el.as_parameters()
            center_x = center[0]
            center_y = center[1]
            ellipses.append([center_x, center_y, width, height, phi])
        else:
            ellipses.append([np.nan, np.nan, np.nan, np.nan, np.nan])

        caudal_edge = [
            float(data['Caudal_edge'][row]),
            float(data['Caudal_edge.1'][row])
        ]
        rostral_edge = [
            float(data['Rostral_edge'][row]),
            float(data['Rostral_edge.1'][row])
        ]
        caudal_edge_ls.append(caudal_edge)
        rostral_edge_ls.append(rostral_edge)


    # ellipse_df = pd.DataFrame(columns=['center_x', 'center_y', 'width', 'height', 'phi'], data=ellipses)
    # a = np.array(ellipse_df['height'][:])
    # b = np.array(ellipse_df['width'][:])
    # ellipse_size_per_frame = a * b * math.pi
    # ellipse_df['ellipse_size'] = ellipse_size_per_frame
    ellipse_df = pd.Dataframe({
        'rostral_edge':rostral_edge_ls,
        'caudal_edge': caudal_edge_ls
    })
    

    print(f'\n ellipses calculation complete')
    
    ellipse_df[['caudal_edge_x', 'caudal_edge_y']] = pd.DataFrame(ellipse_df['caudal_edge'].tolist(), index=ellipse_df.index)
    ellipse_df[['rostral_edge_x', 'rostral_edge_y']] = pd.DataFrame(ellipse_df['rostral_edge'].tolist(), index=ellipse_df.index)
    
    return ellipse_df

def get_pixel_distance(df):
    distances = np.sqrt((df['caudal_edge_x'] - df['rostral_edge_x'])**2 + 
                        (df['caudal_edge_y'] - df['rostral_edge_y'])**2)
    
    mean_distance = np.nanmean(distances)
    std_distance = np.nanstd(distances)
    # Shapiro-Wilk Test
    shapiro_test = stats.shapiro(distances)
    print(f"Shapiro-Wilk Test: Statistic={shapiro_test.statistic}, p-value={shapiro_test.pvalue}")
    
    # Kolmogorov-Smirnov Test
    ks_test = stats.kstest(distances, 'norm', args=(mean_distance, std_distance))
    print(f"Kolmogorov-Smirnov Test: Statistic={ks_test.statistic}, p-value={ks_test.pvalue}")
    
    median_distance = np.median(distances)
    iqr_distance = stats.iqr(distances)
    print(f"Median Distance: {median_distance}")
    print(f"IQR: {iqr_distance}")
    print(f'mean = {mean_distance}')
    print(f'std = {std_distance}')
    return median_distance

R_pix_distance_dict = {}
L_pix_distance_dict = {}

for block in block_collection:
    print(f'working on {block}')
    pl = [i for i in os.listdir(block.r_e_path) if 'DLC' in i and '.csv' in i]
    if len(pl) > 1:
        pl = [i for i in pl if 'filtered' in i][0]
    else:
        pl = pl[0]
    R_csv  = pd.read_csv(block.r_e_path / pl, header=1)
    
    pl = [i for i in os.listdir(block.l_e_path) if 'DLC' in i and '.csv' in i]
    if len(pl) > 1:
        pl = [i for i in pl if 'filtered' in i][0]
    else:
        pl = pl[0]
    L_csv  = pd.read_csv(block.l_e_path / pl, header=1)
    R_ellipse_df = eye_tracking_analysis(R_csv,0.998)
    print('working on the right eye')
    R_pixel_distance = get_pixel_distance(R_ellipse_df)    
    L_ellipse_df = eye_tracking_analysis(L_csv,0.998)
    print('working on the left eye')
    L_pixel_distance = get_pixel_distance(L_ellipse_df)
    R_pix_distance_dict[block.block_num] = R_pixel_distance
    L_pix_distance_dict[block.block_num] = L_pixel_distance
    

In [362]:
def load_eye_data_2d_w_rotation_matrix(block):
    """
    This function checks if the eye dataframes and rotation dict object exist, then imports them
    :param block: The current blocksync class with verifiec re/le dfs
    :return: None
    """
    try:
        block.left_eye_data = pd.read_csv(block.analysis_path / 'left_eye_data.csv', index_col=0, engine='python')
        block.right_eye_data = pd.read_csv(block.analysis_path / 'right_eye_data.csv', index_col=0, engine='python')
    except FileNotFoundError:
        print('eye_data files not found, run the pipeline!')
        return

    try:
        with open(block.analysis_path / 'rotate_eye_data_params.pkl', 'rb') as file:
            rotation_dict = pickle.load(file)
            block.left_rotation_matrix = rotation_dict['left_rotation_matrix']
            block.right_rotation_matrix = rotation_dict['right_rotation_matrix']
            block.left_rotation_angle = rotation_dict['left_rotation_angle']
            block.right_rotation_angle = rotation_dict['right_rotation_angle']
    except FileNotFoundError:
        print('No rotation matrix file, create it')



In [363]:
for block in block_collection:
    block.parse_open_ephys_events()
    block.get_eye_brightness_vectors()
    block.synchronize_block()
    block.create_eye_brightness_df(threshold_value=20)

    # if the code fails here, go to manual synchronization
    block.import_manual_sync_df()
    block.read_dlc_data()
    block.calibrate_pixel_size(10)
    load_eye_data_2d_w_rotation_matrix(block) #should be integrated again... later
    
    # calibrate pupil diameter:

    block.left_eye_data['pupil_diameter_pixels'] = block.left_eye_data.major_ax * 2 * np.pi
    block.right_eye_data['pupil_diameter_pixels'] = block.right_eye_data.major_ax * 2 * np.pi
    block.left_eye_data['pupil_diameter'] = block.left_eye_data['pupil_diameter_pixels'] * block.L_pix_size
    block.right_eye_data['pupil_diameter'] = block.right_eye_data['pupil_diameter_pixels'] * block.R_pix_size

running parse_open_ephys_events...
block 026 has a parsed events file, reading...
Getting eye brightness values for block 026...
Found an existing file!
Eye brightness vectors generation complete.
blocksync_df loaded from analysis folder
eye_brightness_df loaded from analysis folder
eye dataframes loaded from analysis folder
got the calibration values from the analysis folder


In [374]:
block.final_sync_df.columns

Index(['Unnamed: 0.1', 'Arena_TTL', 'Arena_frame', 'L_eye_frame',
       'R_eye_frame', 'L_values', 'R_values'],
      dtype='object')

In [366]:
block.left_eye_data

,OE_timestamp,eye_frame,ms_axis,center_x,center_y,ellipse_phi,width,height,major_ax,minor_ax,ratio,xgb_theta,xgb_phi,phi,theta,pupil_diameter_pixels,pupil_diameter
0,565951.0,1.0,28297.55,444.632190,206.829264,83.016304,45.335549,32.726528,45.335549,32.726528,1.385284,NaN,NaN,41.484692,-28.411240,284.851657,12.818325
1,566291.0,2.0,28314.55,444.501312,206.839827,83.435273,45.316501,32.731487,45.316501,32.731487,1.384493,NaN,NaN,41.484692,-28.411240,284.731971,12.812939
2,566631.0,3.0,28331.55,444.489259,206.843911,83.266029,45.292542,32.789987,45.292542,32.789987,1.381292,NaN,NaN,41.484692,-28.411240,284.581437,12.806165
3,566970.0,4.0,28348.50,444.515784,206.859801,83.509805,45.268229,32.769033,45.268229,32.769033,1.381433,NaN,NaN,41.484692,-28.411240,284.428670,12.799290
4,567310.0,5.0,28365.50,444.444498,206.920830,83.589111,45.316728,32.700692,45.316728,32.700692,1.385803,NaN,NaN,41.484692,-28.411240,284.733399,12.813003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99203,34261949.0,101272.0,1713097.45,412.843225,200.260902,66.957846,46.307113,37.235502,46.307113,37.235502,1.243628,NaN,NaN,39.517727,-33.517628,290.956173,13.093028
99204,34262289.0,101273.0,1713114.45,412.871902,200.210474,66.951497,46.246748,37.140492,46.246748,37.140492,1.245184,NaN,NaN,39.517727,-33.517628,290.576885,13.075960
99205,34262628.0,101274.0,1713131.40,412.912947,200.210626,66.240158,46.229480,37.094161,46.229480,37.094161,1.246274,NaN,NaN,39.518051,-33.517628,290.468388,13.071077
99206,34262968.0,101275.0,1713148.40,413.046975,200.175398,66.738613,46.132734,36.995682,46.132734,36.995682,1.246976,NaN,NaN,39.518051,-33.517628,289.860518,13.043723


In [375]:
block.left_eye_data

,OE_timestamp,eye_frame,ms_axis,center_x,center_y,ellipse_phi,width,height,major_ax,minor_ax,ratio,xgb_theta,xgb_phi,phi,theta,pupil_diameter_pixels,pupil_diameter
0,565951.0,1.0,28297.55,444.632190,206.829264,83.016304,45.335549,32.726528,45.335549,32.726528,1.385284,NaN,NaN,41.484692,-28.411240,284.851657,12.818325
1,566291.0,2.0,28314.55,444.501312,206.839827,83.435273,45.316501,32.731487,45.316501,32.731487,1.384493,NaN,NaN,41.484692,-28.411240,284.731971,12.812939
2,566631.0,3.0,28331.55,444.489259,206.843911,83.266029,45.292542,32.789987,45.292542,32.789987,1.381292,NaN,NaN,41.484692,-28.411240,284.581437,12.806165
3,566970.0,4.0,28348.50,444.515784,206.859801,83.509805,45.268229,32.769033,45.268229,32.769033,1.381433,NaN,NaN,41.484692,-28.411240,284.428670,12.799290
4,567310.0,5.0,28365.50,444.444498,206.920830,83.589111,45.316728,32.700692,45.316728,32.700692,1.385803,NaN,NaN,41.484692,-28.411240,284.733399,12.813003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99203,34261949.0,101272.0,1713097.45,412.843225,200.260902,66.957846,46.307113,37.235502,46.307113,37.235502,1.243628,NaN,NaN,39.517727,-33.517628,290.956173,13.093028
99204,34262289.0,101273.0,1713114.45,412.871902,200.210474,66.951497,46.246748,37.140492,46.246748,37.140492,1.245184,NaN,NaN,39.517727,-33.517628,290.576885,13.075960
99205,34262628.0,101274.0,1713131.40,412.912947,200.210626,66.240158,46.229480,37.094161,46.229480,37.094161,1.246274,NaN,NaN,39.518051,-33.517628,290.468388,13.071077
99206,34262968.0,101275.0,1713148.40,413.046975,200.175398,66.738613,46.132734,36.995682,46.132734,36.995682,1.246976,NaN,NaN,39.518051,-33.517628,289.860518,13.043723


In [382]:
block.final_sync_df[['Arena_TTL','Arena_frame']]

,Arena_TTL,Arena_frame
0,565951.0,913.0
1,566291.0,914.0
2,566631.0,915.0
3,566970.0,916.0
4,567310.0,917.0
...,...,...
99203,34261949.0,100116.0
99204,34262289.0,100117.0
99205,34262628.0,100118.0
99206,34262968.0,100119.0


In [380]:
block.final_sync_df.head()

,Unnamed: 0.1,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,0,565951.0,913.0,1.0,NaN,-2.646521,NaN
1,1,566291.0,914.0,2.0,NaN,-2.646440,NaN
2,2,566631.0,915.0,3.0,NaN,-2.646465,NaN
3,3,566970.0,916.0,4.0,NaN,-2.641369,NaN
4,4,567310.0,917.0,5.0,2.0,-2.641731,0.554543


In [392]:

def create_sync_dataframe(block):
    """
    Constructs a synchronization dataframe for video synchronization.

    Parameters:
        block: BlockSync object containing synchronization data and eye data.

    Returns:
        pd.DataFrame: A dataframe with integrated synchronization data for arena and eye videos.
    """
    # Start with the final_sync_df
    try:
        sync_df = block.final_sync_df[['Arena_TTL', 'Arena_frame']].copy()
        sync_df.rename(columns={'Arena_TTL': 'OE_timestamp'}, inplace=True)
    except KeyError:
        sync_df = block.final_sync_df[['OE_timestamp','Arena_frame']].copy()

    # Merge with left eye data
    left_eye_data = block.left_eye_data[['OE_timestamp', 'eye_frame']].rename(columns={'eye_frame': 'L_eye_frame'})
    sync_df = pd.merge(sync_df, left_eye_data, on='OE_timestamp', how='left')

    # Merge with right eye data
    right_eye_data = block.right_eye_data[['OE_timestamp', 'eye_frame']].rename(columns={'eye_frame': 'R_eye_frame'})
    sync_df = pd.merge(sync_df, right_eye_data, on='OE_timestamp', how='left')

    return sync_df

# Example Usage
sync_df = create_sync_dataframe(block)

In [391]:
block.final_sync_df

,Unnamed: 0.1,OE_timestamp,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,0,565951.0,913.0,1.0,NaN,-2.646521,NaN
1,1,566291.0,914.0,2.0,NaN,-2.646440,NaN
2,2,566631.0,915.0,3.0,NaN,-2.646465,NaN
3,3,566970.0,916.0,4.0,NaN,-2.641369,NaN
4,4,567310.0,917.0,5.0,2.0,-2.641731,0.554543
...,...,...,...,...,...,...,...
99203,99203,34261949.0,100116.0,101272.0,101271.0,0.072080,-1.185376
99204,99204,34262289.0,100117.0,101273.0,101272.0,0.072150,-1.185574
99205,99205,34262628.0,100118.0,101274.0,101273.0,0.072569,-1.186444
99206,99206,34262968.0,100119.0,101275.0,101274.0,0.075081,-1.186171


In [387]:
block.final_sync_df

,Unnamed: 0.1,Arena_TTL,Arena_frame,L_eye_frame,R_eye_frame,L_values,R_values
0,0,565951.0,913.0,1.0,NaN,-2.646521,NaN
1,1,566291.0,914.0,2.0,NaN,-2.646440,NaN
2,2,566631.0,915.0,3.0,NaN,-2.646465,NaN
3,3,566970.0,916.0,4.0,NaN,-2.641369,NaN
4,4,567310.0,917.0,5.0,2.0,-2.641731,0.554543
...,...,...,...,...,...,...,...
99203,99203,34261949.0,100116.0,101272.0,101271.0,0.072080,-1.185376
99204,99204,34262289.0,100117.0,101273.0,101272.0,0.072150,-1.185574
99205,99205,34262628.0,100118.0,101274.0,101273.0,0.072569,-1.186444
99206,99206,34262968.0,100119.0,101275.0,101274.0,0.075081,-1.186171


In [388]:
def compare_eye_frames(new_sync_df, final_sync_df):
    """
    Compares the L_eye_frame and R_eye_frame columns between new_sync_df and final_sync_df.

    Parameters:
        new_sync_df (pd.DataFrame): The newly created synchronization dataframe.
        final_sync_df (pd.DataFrame): The original synchronization dataframe.

    Returns:
        dict: A dictionary with mismatches and summary statistics.
    """
    final_sync_df.rename(columns={'Arena_TTL': 'OE_timestamp'}, inplace=True)
    # Merge the dataframes on OE_timestamp for alignment
    comparison_df = pd.merge(
        new_sync_df[['OE_timestamp', 'L_eye_frame', 'R_eye_frame']],
        final_sync_df[['OE_timestamp', 'L_eye_frame', 'R_eye_frame']],
        on='OE_timestamp',
        suffixes=('_new', '_final'),
        how='inner'
    )

    # Compare the columns
    comparison_df['L_eye_mismatch'] = comparison_df['L_eye_frame_new'] != comparison_df['L_eye_frame_final']
    comparison_df['R_eye_mismatch'] = comparison_df['R_eye_frame_new'] != comparison_df['R_eye_frame_final']

    # Identify mismatches
    l_eye_mismatches = comparison_df[comparison_df['L_eye_mismatch']]
    r_eye_mismatches = comparison_df[comparison_df['R_eye_mismatch']]

    # Summary statistics
    mismatch_summary = {
        'total_frames': len(comparison_df),
        'L_eye_mismatches': len(l_eye_mismatches),
        'R_eye_mismatches': len(r_eye_mismatches),
        'L_eye_mismatch_percentage': len(l_eye_mismatches) / len(comparison_df) * 100,
        'R_eye_mismatch_percentage': len(r_eye_mismatches) / len(comparison_df) * 100,
    }

    return {
        'comparison_df': comparison_df,
        'l_eye_mismatches': l_eye_mismatches,
        'r_eye_mismatches': r_eye_mismatches,
        'mismatch_summary': mismatch_summary
    }

# Example Usage
results = compare_eye_frames(sync_df, block.final_sync_df)

# Display summary of mismatches
print("Mismatch Summary:")
for key, value in results['mismatch_summary'].items():
    print(f"{key}: {value}")

# Optionally, save the comparison dataframe to a CSV for detailed analysis
results['comparison_df'].to_csv("eye_frame_comparison.csv", index=False)


Mismatch Summary:
total_frames: 99208
L_eye_mismatches: 20951
R_eye_mismatches: 10525
L_eye_mismatch_percentage: 21.118256592210304
R_eye_mismatch_percentage: 10.609023465849528


In [ ]:
def save_synchronized_videos(sync_df, arena_vid_left, arena_vid_top, left_vid, right_vid, indices, output_file, fps=60, duration=5, separator_duration=2):
    """
    Saves synchronized frames from four video sources into a single concatenated video.

    Parameters:
        sync_df (pd.DataFrame): A dataframe with columns ['Arena_frame', 'L_eye_frame', 'R_eye_frame'].
        arena_vid_left (str): Path to the left arena video file.
        arena_vid_top (str): Path to the top arena video file.
        left_vid (str): Path to the left eye video file.
        right_vid (str): Path to the right eye video file.
        indices (list): List of starting indices for segments to save.
        output_file (str): Path to the output video file.
        fps (int): Frames per second for the output video.
        duration (int): Duration of each mini-video segment in seconds.
        separator_duration (int): Duration of black screen separator in seconds.
    """
    # Open video capture objects
    arena_left_cap = cv2.VideoCapture(arena_vid_left)
    arena_top_cap = cv2.VideoCapture(arena_vid_top)
    left_cap = cv2.VideoCapture(left_vid)
    right_cap = cv2.VideoCapture(right_vid)

    if not (arena_left_cap.isOpened() and arena_top_cap.isOpened() and left_cap.isOpened() and right_cap.isOpened()):
        print("Error: Could not open one or more video files.")
        return

    # Get video properties
    frame_width = int(arena_left_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(arena_left_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    black_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

    # Define output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_size = (frame_width * 2, frame_height * 2)
    out = cv2.VideoWriter(output_file, fourcc, fps, output_size)

    # Function to resize frames while keeping aspect ratio
    def resize_frame(frame, scale, canvas):
        new_w, new_h = int(frame.shape[1] * scale), int(frame.shape[0] * scale)
        resized = cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_AREA)
        canvas.fill(0)
        canvas[(canvas.shape[0] - new_h) // 2:(canvas.shape[0] - new_h) // 2 + new_h,
               (canvas.shape[1] - new_w) // 2:(canvas.shape[1] - new_w) // 2 + new_w] = resized
        return canvas

    # Function to concatenate frames
    def concatenate_frames(frames):
        top_row = np.hstack((frames[0], frames[1]))  # Arena left and top
        bottom_row = np.hstack((frames[2], frames[3]))  # Left and right eye
        return np.vstack((top_row, bottom_row))

    # Process each segment
    for start_idx in indices:
        end_idx = start_idx + int(duration * fps)

        for idx in range(start_idx, end_idx):
            if idx >= len(sync_df):
                break

            row = sync_df.iloc[idx]
            arena_left_frame, arena_top_frame = row['Arena_frame'], row['Arena_frame']
            left_frame, right_frame = row['L_eye_frame'], row['R_eye_frame']

            # Retrieve frames
            def get_frame(cap, frame_idx):
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = cap.read()
                return frame if ret else black_frame

            arena_left_img = get_frame(arena_left_cap, arena_left_frame)
            arena_top_img = get_frame(arena_top_cap, arena_top_frame)
            left_img = get_frame(left_cap, left_frame)
            right_img = get_frame(right_cap, right_frame)

            # Prepare individual canvases
            arena_canvas_left = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
            arena_canvas_top = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
            eye_canvas_left = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
            eye_canvas_right = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)

            # Resize frames to match dimensions
            arena_left_img = resize_frame(arena_left_img, 1.0, arena_canvas_left)
            arena_top_img = resize_frame(arena_top_img, 1.0, arena_canvas_top)
            left_img = resize_frame(left_img, 1.0, eye_canvas_left)
            right_img = resize_frame(right_img, 1.0, eye_canvas_right)

            # Concatenate resized frames
            concatenated_frame = concatenate_frames([arena_left_img, arena_top_img, left_img, right_img])
            out.write(concatenated_frame)

        # Add separator (black screen)
        for _ in range(int(separator_duration * fps)):
            separator_frame = np.vstack((np.hstack((black_frame, black_frame)), np.hstack((black_frame, black_frame))))
            out.write(separator_frame)

    # Release resources
    arena_left_cap.release()
    arena_top_cap.release()
    left_cap.release()
    right_cap.release()
    out.release()


# Example usage
arena_vid_left = r'Z:\Nimrod\experiments\PV_62\2023_04_27\block_026\arena_videos\videos\left_20230427T122154.mp4'
arena_vid_top = r'Z:\Nimrod\experiments\PV_62\2023_04_27\block_026\arena_videos\videos\top_20230427T122154.mp4'
left_vid = block.le_videos[0]
right_vid = block.re_videos[0]
sync_df = block.final_sync_df
indices = [3420, 3639,8820,12336,14856,18576,22626,25656,29646,33066,47916,49446,52476,60516,63996,65556,67145,75066]  # Start indices for segments
output_file = "output_video.mp4"

save_synchronized_videos(sync_df, arena_vid_left, arena_vid_top, left_vid, right_vid, indices, output_file)